# 02 - Agentic RAG Cryptography Assistant

## Requirements
These are the required _Libraries_ and _Environment Variables_ for this notebook


### Libraries Required


In [ ]:
# for setting up Jupyter widgets and notebook features
%conda install conda-forge::ipywidgets --update-deps --force-reinstall
%conda install conda-forge::ipykernel --update-deps --force-reinstall


- [LangSmith](https://docs.langchain.com/langsmith/home)
- [LangChain](https://reference.langchain.com/python/langchain/langchain)
- [LangChain Core](https://reference.langchain.com/python/langchain_core)
- [LangChain Groq Model Provider](https://reference.langchain.com/python/integrations/langchain_groq)
- [Chroma Vector Database](https://docs.trychroma.com/docs/overview/introduction)
- [Sentence Transformers](https://www.sbert.net)

In [ ]:
%conda install conda-forge::langsmith
%conda install conda-forge::langchain --update-deps --force-reinstall
%conda install conda-forge::langchain-core --update-deps --force-reinstall
%conda install conda-forge::langchain-groq

%conda install conda-forge::chromadb

%conda install conda-forge::pyarrow
%conda install conda-forge::datasets
%conda install conda-forge::sentence-transformers


### Variables Required

| Token Name                     | `.env` Name          | Where to Get / Setting Value                                                      |                                                                                              Reference |
| :----------------------------- | :------------------- | :-------------------------------------------------------------------------------- | -----------------------------------------------------------------------------------------------------: |
| Groq API Key                   | `GROQ_API_KEY`       | [Groq Console](https://console.groq.com/keys)                                     |                                              [Groq API Docs](https://console.groq.com/docs/quickstart) |
| LangSmith API Key              | `LANGSMITH_API_KEY`  | [LangSmith Settings](https://smith.langchain.com/settings)                        |                                   [LangSmith API Reference](https://docs.langchain.com/langsmith/home) |
| LangSmith Tracing              | `LANGSMITH_TRACING`  | A Boolean, set the value to `true` or `false` to enable or disable logging traces | [LangSmith Observability API Reference](https://docs.langchain.com/langsmith/observability-quickstart) |
| LangSmith Endpoint             | `LANGSMITH_ENDPOINT` | The LangSmith Endpoint to log the traces (<https://api.smith.langchain.com>)      | [LangSmith Observability API Reference](https://docs.langchain.com/langsmith/observability-quickstart) |
| LangSmith Project Name         | `LANGSMITH_PROJECT`  | The name of the project to log the traces under                                   | [LangSmith Observability API Reference](https://docs.langchain.com/langsmith/observability-quickstart) |
| Hugging Face User Access Token | `HF_TOKEN`           | [Hugging Face Settings](https://huggingface.co/settings/tokens)                   |                                                       [Hugging Face Docs](https://huggingface.co/docs) |


In [ ]:
from pathlib import Path
import sys

ROOT = Path().resolve().parent.parent
sys.path.append(str(ROOT))


In [ ]:
from utils import (
    set_env_variables,
)

ENV_FILE = ROOT / ".env"

set_env_variables(env_file=ENV_FILE)


## Actual Shenanigans


In [2]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_groq import ChatGroq
from langchain.messages import HumanMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter

from sentence_transformers import SentenceTransformer

import chromadb


### Initializing Chroma Collection


#### Creating Documents

In [3]:
documents = [
    {
        "title": "Multi-key fully homomorphic encryption from NTRU and (R)LWE with faster bootstrapping",
        "source": "Theoretical Computer Science 968 (2023) 114026",
        "content": """This paper proposes a new Multi-Key Fully Homomorphic Encryption (MKFHE) scheme based on NTRU and (R)LWE, aimed at improving bootstrapping performance over existing LWE-based schemes. The authors adapt the bootstrapping algorithm from Chen et al. to the NTRU setting and introduce an efficient conversion method from multi-key NTRU to multi-key RLWE ciphertexts. This approach, with carefully chosen parameters, is designed to be secure against sublattice attacks that have compromised previous NTRU-based schemes. Experimental results from a proof-of-concept implementation demonstrate that their scheme's gate bootstrapping for two parties is 5.3 times faster than the state-of-the-art, although its performance does not scale well to a larger number of parties. The scheme is presented as practical for applications like oblivious neural network inference.""",
    },
    {
        "title": "Research on Noise Management Technology for Fully Homomorphic Encryption",
        "source": "IEEE Access, vol. 12, 2024, pp. 135564-135576",
        "content": """This paper provides a comprehensive analysis of noise management techniques in fully homomorphic encryption (FHE), a critical aspect for enabling arbitrary computations on encrypted data. The authors classify existing methods into categories such as bootstrapping, bit expansion, and scaling, evaluating the principles, strengths, and weaknesses of each. A key focus is on the application and advantages of the gadget matrix for noise management. The paper analyzes noise growth and parameter sizes in major FHE schemes to propose optimization strategies. Finally, it presents an optimized version of the Gentry-Sahai-Waters (GSW) scheme that uses a gadget matrix. This optimization is shown to reduce the size of both the private key and the ciphertext by a factor of [log q], resulting in better overall complexity compared to the original GSW scheme.""",
    },
    {
        "title": "Concise and Efficient Multi-Identity Fully Homomorphic Encryption Scheme",
        "source": "IEEE Access, vol. 12, 2024, pp. 49640-49653",
        "content": """
        This paper introduces a concise and efficient multi-identity based fully homomorphic encryption (MIBFHE) scheme to address the complexity and computational overhead of existing solutions. The authors combine multi-key fully homomorphic encryption (MKFHE) with identity-based encryption (IBE) to enable both homomorphic operations and flexible, identity-based access control. The core contribution is a new MKFHE scheme based on the learning with errors (LWE) problem, constructed using a novel "decomposition method." This method allows for the direct generation of extended ciphertexts without needing a fresh ciphertext first. The resulting MIBFHE scheme is proven IND-SID-CPA secure. It features a significantly reduced noise expansion rate and a smaller scale of auxiliary ciphertexts compared to previous work.
        """,
    },
    {
        "title": "Multikey Verifiable Homomorphic Encryption",
        "source": "IEEE Access, vol. 10, 2022, pp. 84761-84775",
        "content": """This paper addresses the challenge of verifying computation results in a multi-client delegated computation scenario. While verifiable homomorphic encryption (VHE) provides a solution for single-user settings, it is insufficient when multiple parties with different keys are involved. To solve this, the authors introduce a new cryptographic primitive called multi-key verifiable homomorphic encryption (MVHE). The paper provides a formalization of MVHE and presents a construction for it. This construction is achieved by combining a multi-key homomorphic encryption (MHE) scheme with another new primitive proposed in the paper: a multi-key homomorphic encrypted authentication scheme. The work extends the security guarantee of verifiable computation to settings with multiple, mutually untrusting data owners.""",
    },
]


#### Embeddings and Database

Initializing the

- [BAAI BGE-SMALL Embedding model](https://huggingface.co/BAAI/bge-small-en-v1.5)
- Chroma [Ephemeral Client](https://docs.trychroma.com/docs/run-chroma/ephemeral-client) and [Collection](https://docs.trychroma.com/docs/collections/manage-collections#embedding-functions) using the BGE embedding model


In [4]:
embedding_model = SentenceTransformer("BAAI/bge-small-en-v1.5")

chroma_client = chromadb.EphemeralClient()

collection = chroma_client.get_or_create_collection(
    name="crypto_documents_v1", metadata={"hnsw:space": "cosine"}
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, chunk_overlap=50, separators=["\n\n", ".", " "]
)
chunk_counter = 0
for doc in documents:
    chunks = text_splitter.split_text(doc["content"])
    print(f"  '{doc['title']}' → {len(chunks)} chunks")

    for chunk_idx, chunk in enumerate(chunks):
        embedding = embedding_model.encode([chunk])[0].tolist()

        collection.add(
            ids=[f"doc_{chunk_counter}"],
            documents=[chunk],
            embeddings=[embedding],
            metadatas=[
                {
                    "source": doc["source"],
                    "title": doc["title"],
                    "chunk_index": chunk_idx,
                }
            ],
        )

        chunk_counter += 1
print(f"Total chunks added to ChromaDB: {chunk_counter}")


  'Multi-key fully homomorphic encryption from NTRU and (R)LWE with faster bootstrapping' → 4 chunks
  'Research on Noise Management Technology for Fully Homomorphic Encryption' → 4 chunks
  'Concise and Efficient Multi-Identity Fully Homomorphic Encryption Scheme' → 4 chunks
  'Multikey Verifiable Homomorphic Encryption' → 4 chunks
Total chunks added to ChromaDB: 16


### Defining Retrieval Tool

In [5]:
@tool
def search_documents(query: str, top_k: int = 3) -> str:
    """Search the document database. Use query parameter for your search term and top_k for number of results (default 3)."""
    try:
        query_embedding = embedding_model.encode([query])[0].tolist()

        results = collection.query(
            query_embeddings=[query_embedding], n_results=top_k if top_k > 0 else 3
        )

        if not results["documents"] or len(results["documents"][0]) == 0:
            return "No relevant documents found. Try a different query."

        formatted_results = []
        for i, (document, metadata, distance) in enumerate(
            zip(
                results["documents"][0],
                results["metadatas"][0],  # type: ignore
                results["distances"][0],  # type: ignore
            )
        ):
            similarity_score = 1 - distance
            formatted_results.append(
                f"\n--- Result {i + 1} (Relevance: {similarity_score:.2%}) ---\n"
                f"Source: {metadata['title']} ({metadata['source']})\n"
                f"Content: {document}\n"
            )

        return "".join(formatted_results)

    except Exception as e:
        return f"Error searching documents: {str(e)}. Please try again."


### Initializing Model and Agent

<div class="alert alert-info">

ℹ️ **Note**

Head over to the [Rate Limits section](https://console.groq.com/docs/rate-limits) in the Groq docs to learn more about the models and their rate limits!

_ps: I hit the ratelimit for the `llama-3.3-70b-versatile` which is why i switched to `llama-3.1-8b-instant` 😅_

</div>

In [6]:
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0, max_tokens=2048)

# i hit rate limit 😅
# llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0, max_tokens=2048)

system_prompt = """You are a helpful cryptography information assistant. You have access to a database of cryptography documents.

When answering questions:
1. ALWAYS search the document database first using the search_documents tool with a query parameter
2. The search_documents tool takes two parameters: query (required) and top_k (optional, default 3)
3. CITE the sources you find
4. If the documents don't contain relevant information, say so
5. NEVER make up information
6. Include relevant information from multiple documents when appropriate
7. For drug interactions or safety concerns, be especially careful and cite sources

Your goal is to provide accurate, well-sourced cryptography information."""

agent = create_agent(model=llm, tools=[search_documents], system_prompt=system_prompt)


### Running the Agent

#### Demo the agent with example queries

In [7]:
print("\n" + "=" * 70)
print("Demo")
print("=" * 70)

demo_queries = [
    "Besides keeping data private, what other major security concern arises when outsourcing computation to an untrusted third party, and what cryptographic property can address it?",
    "Why is noise an inherent and fundamental problem in most homomorphic encryption schemes?",
    "What is bootstrapping in the context of FHE, and what problem does it solve?",
]

for i, query in enumerate(demo_queries, 1):
    print(f"\n[Query {i}] {query}")

    try:
        result = agent.invoke({"messages": [HumanMessage(content=query)]})

        answer = result["messages"][-1].content
        print(f"Agent>\n{answer}\n")

    except Exception as e:
        print(f"Error: {e}\n")



Demo

[Query 1] Besides keeping data private, what other major security concern arises when outsourcing computation to an untrusted third party, and what cryptographic property can address it?


Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=dcd17b9d-580e-4d8d-b8bd-664c56c2eac7,id=dcd17b9d-580e-4d8d-b8bd-664c56c2eac7; trace=dcd17b9d-580e-4d8d-b8bd-664c56c2eac7,id=8fe8e6f2-aefa-40a6-b8f9-1ad8f158dfff; trace=dcd17b9d-580e-4d8d-b8bd-664c56c2eac7,id=9d72a4fa-18fd-46fe-837e-970035d1b3ec
Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=dcd17b9d-580e-

Agent>
Another major security concern when outsourcing computation to an untrusted third party is the potential for data tampering or manipulation. This can occur when the third party modifies the input data or the computation results without the knowledge or consent of the data owner.

The cryptographic property that can address this concern is verifiable computation, specifically verifiable homomorphic encryption (VHE). VHE allows the data owner to verify that the computation was performed correctly and that the results have not been tampered with, even if the computation is performed by an untrusted third party.

In particular, multikey verifiable homomorphic encryption (MVHE) can address the challenge of verifying computation results in a multi-client delegated computation scenario, where multiple parties with different keys are involved. This ensures that the computation results are accurate and trustworthy, even in the presence of multiple, mutually untrusting data owners.


[Que

Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=dcd17b9d-580e-4d8d-b8bd-664c56c2eac7,id=9e17816d-e0f5-45bb-ac10-86a8ee6c89f8; trace=dcd17b9d-580e-4d8d-b8bd-664c56c2eac7,id=8136bc87-0c5d-414b-9bba-2636af90b517; trace=dcd17b9d-580e-4d8d-b8bd-664c56c2eac7,id=429a808e-c59b-4a9f-9428-d66a8a3a147f; trace=dcd17b9d-580e-4d8d-b8bd-664c56c2eac7,id=cfb09ede-21a6-4026-9276-2ebec261f71a; trace=dcd17b9d-580e-4d8d-b8bd-664c56c2eac7,id=cfb09ede-21a6-4026-9276-2ebec261f71a; trace=dcd17b9d-580e-4d8d-b8bd-664c56c2eac7,id=429a808e-c59b-4a9f-9428-d66a8a3a147f; trace=dcd17b9d-580e-4d8d-b8bd-664c56c2eac7,id=f4279b75-7c48-4b13-8474-dcd8d6dbface; trace=dcd17b9d-580e-4d8d-b8bd-664c56c2eac7

Agent>
Bootstrapping in the context of Fully Homomorphic Encryption (FHE) is a technique used to reduce the noise growth in FHE schemes, allowing for more efficient and scalable computations. It solves the problem of noise accumulation, which is a major challenge in FHE, by refreshing the ciphertexts to a noise-free state.

According to the results from the document database, bootstrapping in FHE is a crucial aspect of improving the performance and scalability of FHE schemes. The results highlight the importance of noise management in FHE and the need for efficient bootstrapping techniques.

Sources:

* Multi-key fully homomorphic encryption from NTRU and (R)LWE with faster bootstrapping (Theoretical Computer Science 968 (2023) 114026)
* Research on Noise Management Technology for Fully Homomorphic Encryption (IEEE Access, vol. 12, 2024, pp. 135564-135576)
* Concise and Efficient Multi-Identity Fully Homomorphic Encryption Scheme (IEEE Access, vol. 12, 2024, pp. 49640-49653)



Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=3399a10c-9972-47b3-a245-512cc4e82acd,id=4f719b26-87bd-45e6-b33b-04953156a831; trace=3399a10c-9972-47b3-a245-512cc4e82acd,id=206fe80a-7d80-4a83-aab3-d1a2721e2e23; trace=3399a10c-9972-47b3-a245-512cc4e82acd,id=3399a10c-9972-47b3-a245-512cc4e82acd
Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=31c

#### Interactive RAG System

In [8]:
def run_crypto_rag_agent(query: str):
    """Run the agentic RAG system in interactive mode."""

    try:
        print("\nSearching documents and thinking...")

        try:
            result = agent.invoke({"messages": [HumanMessage(content=query)]})

            final_answer = result["messages"][-1].content
            print(f"\nAssistant>\n{final_answer}")

        except Exception as agent_error:
            error_msg = f"Error processing query: {str(agent_error)}"
            print(f"\nerror: {error_msg}")
            print("Tip: Try rephrasing your question or use simpler language.")

    except Exception as e:
        print(f"\nUnexpected error: {str(e)}")
        print("Please try again.")


In [9]:
import ipywidgets as widgets
from IPython.display import display

text = widgets.Text(placeholder="Ask away!")
send = widgets.Button(description="Send")
out = widgets.Output()
box_layout = widgets.Layout(
    overflow="scroll hidden",
    width="100%",
    flex_flow="row",
    display="flex",
)

print("\n" + "=" * 70)
print("Interactive Mode")
print("=" * 70)

print("\nI have cryptography papers. Ask me anything about Homomorphic Cryptography.")
print("\nExamples:")
print("  - 'What is FHE?'")
print(
    "  - 'What are some of the key trade-offs involved in designing practical FHE schemes?'"
)
print(
    "  - 'What is the conceptual purpose of using different mathematical tools in FHE constructions?'"
)
print(
    "  - 'What is the role of bootstrapping in achieving fully homomorphic encryption, and how does it solve the noise problem?'"
)
print("=" * 70)

query_count = 0


def on_click(b):
    global query_count
    query = text.value.strip()
    if not query:
        return
    text.value = ""
    query_count += 1

    with out:
        try:
            print("\n" + "=" * 70)
            print(f"\n[Query {query_count}] {query}")
            run_crypto_rag_agent(query)
        except Exception as e:
            print("Error:", e)


send.on_click(on_click)
display(
    widgets.VBox(
        [text, send, widgets.Box([widgets.Label("Responses:"), out], layout=box_layout)]
    )
)



Interactive Mode

I have cryptography papers. Ask me anything about Homomorphic Cryptography.

Examples:
  - 'What is FHE?'
  - 'What are some of the key trade-offs involved in designing practical FHE schemes?'
  - 'What is the conceptual purpose of using different mathematical tools in FHE constructions?'
  - 'What is the role of bootstrapping in achieving fully homomorphic encryption, and how does it solve the noise problem?'
